In [1]:
# Load packages
import os
import numpy as np
import pandas as pd
import tensorflow as tf

tf.__version__

'1.13.1'

In [2]:
# Load data
data = pd.read_csv("/Users/nityansuman/__data__/iris-dataset.csv")

print(data.shape)

print(data.columns)

(150, 6)
Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')


In [3]:
# View
data.head(3)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa


In [4]:
# Get class labels
class_labels = data.Species.unique()

class_labels

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [5]:
# Categorize labels
class_label_mapping = dict(zip(class_labels, range(len(class_labels))))

class_label_mapping

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

In [6]:
# Map class labels
data.Species = data.Species.apply(lambda x: class_label_mapping[x])

data.head(3)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0


In [7]:
# Select labels in [0, 1]
data = data[data.Species < 2]

data.shape

(100, 6)

In [8]:
# View
data.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


In [9]:
# Get class labels from the updated data
data.Species.unique()

array([0, 1])

In [10]:
# Load features into a numpy array
features = np.array(data.iloc[:, 1:5])

features.shape

(100, 4)

In [11]:
# Load class labels into a numpy array
labels = np.array(data.iloc[:, [-1]])

labels.shape

(100, 1)

In [12]:
# View
print(features[0], labels[0])

print(features[1], labels[1])

[5.1 3.5 1.4 0.2] [0]
[4.9 3.  1.4 0.2] [0]


In [13]:
# Generate random indexes
random_indices = np.random.rand(len(features)) <= 0.2

# Split data into train and validation
train_x, val_x = features[~random_indices], features[random_indices]
train_y, val_y = labels[~random_indices], labels[random_indices]

print(train_x.shape, train_y.shape)

print(val_x.shape, val_y.shape)

(83, 4) (83, 1)
(17, 4) (17, 1)


### Logistic regression model

In [14]:
# Define placeholders for taking data input
x = tf.placeholder(dtype=tf.float32, shape=(None, features.shape[1]), name="feature_x")
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="target_y")

# Define weights and bias
w = tf.Variable(tf.random_normal(shape=(1, features.shape[1]), dtype=tf.float32, mean=0.0, stddev=0.1))
b = tf.Variable(tf.zeros(shape=(1, 1), dtype=tf.float32))

# Core logistic model
logistic_model = tf.sigmoid(tf.add(tf.matmul(x, tf.transpose(w)), b))

# oss - binary cross-entropy
loss_function = tf.reduce_mean(-(y*tf.log(logistic_model) + (1 - y)*tf.log(1 - logistic_model)))

# Optimizer to minimize loss
trainer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss_function)

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
# Train model and validate after each epoch
with tf.Session() as sess:
    # Initialize global variables
    sess.run(tf.global_variables_initializer())
    print("Training...")
    for epoch in range(10):
        # For each epoch run the trainer object
        sess.run(trainer, feed_dict={x:train_x, y:train_y})
        # Calculate train and validation loss
        train_loss = sess.run(loss_function, feed_dict={x:train_x, y:train_y})
        val_loss = sess.run(loss_function, feed_dict={x:val_x, y:val_y})
        # Prompt
        print("Step:", epoch, "\tTrain Loss:", train_loss, end="\t")
        print("Val Loss:", val_loss)
    # Get final weights after complete run
    print("Final Weights:", sess.run(w))
    # Get prediction on validation data finally
    predicted_y = sess.run(logistic_model, feed_dict={x:val_x, y:val_y})

Training...
Step: 0 	Train Loss: 0.713304	Val Loss: 0.79438305
Step: 1 	Train Loss: 0.60375005	Val Loss: 0.6487528
Step: 2 	Train Loss: 0.51122075	Val Loss: 0.5136444
Step: 3 	Train Loss: 0.44525996	Val Loss: 0.422934
Step: 4 	Train Loss: 0.38085973	Val Loss: 0.35610744
Step: 5 	Train Loss: 0.32036656	Val Loss: 0.30397192
Step: 6 	Train Loss: 0.27380463	Val Loss: 0.26605093
Step: 7 	Train Loss: 0.23787647	Val Loss: 0.23415956
Step: 8 	Train Loss: 0.20595118	Val Loss: 0.20154338
Step: 9 	Train Loss: 0.17666644	Val Loss: 0.1688931
Final Weights: [[-0.06211819 -0.612569    0.81895185  0.80761826]]


### Analysis

In [16]:
final_prediction = list()

# Iterate over all predictions
for pred in predicted_y:
    # Convert predictions into binary labels
    if pred >= 0.5:
        final_prediction.append(1)
    else:
        final_prediction.append(0)

# Convert list into a numpy array
final_prediction = np.array(final_prediction).reshape((len(final_prediction), 1))

In [17]:
# Assert shape of prediction and ground-truth is same for performance analysis
assert final_prediction.shape == val_y.shape
print(final_prediction.shape, val_y.shape)

(17, 1) (17, 1)


In [18]:
# Compute accuracy
correct_prediciton = 0

for i in range(val_y.shape[0]):
    if final_prediction[i] == val_y[i]:
        correct_prediciton += 1
    else:
        continue

print("Accuracy achieved is {}%.".format(correct_prediciton / val_y.shape[0] * 100))

Accuracy achieved is 100.0%.


### Using manually deifned optimzer instead of inbuilt as before

In [19]:
# Compute gradients
grad_w, grad_b = tf.gradients(xs=[w, b], ys=loss_function)

# Update weighst and baises
new_w = w.assign(w - 0.1*grad_w)
new_b = b.assign(b - 0.1*grad_b)

In [20]:
# Train model and validate after each epoch
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("Training...")
    for epoch in range(10):
        # For each epoch
        # Compute train loss and update weights and biases
        _, _, train_loss = sess.run([new_w, new_b, loss_function], feed_dict={x:train_x, y:train_y})
        # Comnpute validation loss
        val_loss = sess.run(loss_function, feed_dict={x:val_x, y:val_y})
        # Prompt
        print("Step:", epoch, "\tTrain Loss:", train_loss, end="\t")
        print("Val Loss:", val_loss)
    # Get final weights after train
    print("Final Weights:", sess.run(w))
    # Get predictions on validation set
    predicted_y = sess.run(logistic_model, feed_dict={x:val_x, y:val_y})

Training...
Step: 0 	Train Loss: 0.60738117	Val Loss: 0.5808936
Step: 1 	Train Loss: 0.5573544	Val Loss: 0.53519475
Step: 2 	Train Loss: 0.52550936	Val Loss: 0.50505346
Step: 3 	Train Loss: 0.49714863	Val Loss: 0.4758059
Step: 4 	Train Loss: 0.4711988	Val Loss: 0.44940397
Step: 5 	Train Loss: 0.44740018	Val Loss: 0.42522544
Step: 6 	Train Loss: 0.42553926	Val Loss: 0.4030858
Step: 7 	Train Loss: 0.40542427	Val Loss: 0.38276985
Step: 8 	Train Loss: 0.38688323	Val Loss: 0.36409226
Step: 9 	Train Loss: 0.36976215	Val Loss: 0.34688756
Final Weights: [[-0.16149585 -0.19399698  0.5348944   0.28587335]]


**validation loss after same number of epochs is little higher than what we saw earlier. This can happen sinze model satrtes train from a random states.**

### Analysis

In [21]:
final_prediction = list()

# Iterate over all predictions
for pred in predicted_y:
    # Convert predictions into binary labels
    if pred >= 0.5:
        final_prediction.append(1)
    else:
        final_prediction.append(0)

# Convert list into a numpy array
final_prediction = np.array(final_prediction).reshape((len(final_prediction), 1))

In [22]:
# Assert shape of prediction and ground-truth is same for performance analysis
assert final_prediction.shape == val_y.shape
print(final_prediction.shape, val_y.shape)

(17, 1) (17, 1)


In [23]:
# Compute accuracy
correct_prediciton = 0

for i in range(val_y.shape[0]):
    if final_prediction[i] == val_y[i]:
        correct_prediciton += 1
    else:
        continue

print("Accuracy achieved is {}%.".format(correct_prediciton / val_y.shape[0] * 100))

Accuracy achieved is 100.0%.


**Perfect Score!**